#### Prepared for Gabor's Data Analysis

### Data Analysis for Business, Economics, and Policy
by Gabor Bekes and  Gabor Kezdi
 
Cambridge University Press 2021

**[gabors-data-analysis.com ](https://gabors-data-analysis.com/)**

 License: Free to share, modify and use for educational purposes. 
 Not to be used for commercial purposes.

### Chapter 09
**CH09B How stable is the hotel price - distance to center relationship?**

using the hotels-europe dataset

version 1.0 2021-05-05

In [1]:
import os
import sys
import warnings

import numpy as np
import pandas as pd
from mizani.formatters import percent_format
from plotnine import *
import statsmodels.formula.api as smf
from stargazer.stargazer import Stargazer
from IPython.core.display import HTML

warnings.filterwarnings("ignore")


In [2]:
# Current script folder
current_path = os.getcwd()
dirname = current_path.split("da_case_studies")[0]

# location folders
data_in = dirname + "da_data_repo/hotels-europe/clean/"
data_out = dirname + "da_case_studies/ch09-hotels-europe-stability/"
output = dirname + "da_case_studies/ch09-hotels-europe-stability/output/"
func = dirname + "da_case_studies/ch00-tech-prep/"
sys.path.append(func)


In [3]:
# Import the prewritten helper functions
from py_helper_functions import *


In [5]:
# load in clean and tidy data and create workfile
hotels_europe_price = pd.read_csv("/workspaces/codespaces-jupyter/data/hotels-europe_price.csv")
hotels_europe_features = pd.read_csv("/workspaces/codespaces-jupyter/data/hotels-europe_features.csv")

#hotels_europe_price = pd.read_csv("https://osf.io/download/p6tyr/")
#hotels_europe_features = pd.read_csv("https://osf.io/download/utwjs/")


In [6]:
data = pd.merge(hotels_europe_price, hotels_europe_features, on="hotel_id", how="left")

In [7]:
# filter a few cities
data = data.loc[data["city_actual"].isin(["Vienna", "Amsterdam", "Barcelona"])]

In [8]:
data = data.loc[data["accommodation_type"].isin(["Hotel", "Apartment"])]

In [9]:
# drop long stay , 1000E+
data = data[data["nnights"] != 4]
data = data[data["price"] < 1000]

In [10]:
# check for duplicates
data = data.drop_duplicates()

In [11]:
# filter for days
data.loc[(data["month"] == 11) & (data["weekend"] == 0), "date"] = "2017-NOV-weekday"
data.loc[(data["month"] == 11) & (data["weekend"] == 1), "date"] = "2017-NOV-weekend"
data.loc[(data["month"] == 12) & (data["holiday"] == 1), "date"] = "2017-DEC-holiday"
data.loc[(data["month"] == 6) & (data["weekend"] == 1), "date"] = "2018-JUNE-weekend"

In [12]:
data = data[data["date"].notna()]

In [13]:
data["city"].value_counts()

city
Barcelona    1564
Vienna       1326
Amsterdam     830
Name: count, dtype: int64

In [14]:
pd.crosstab(index=data["accommodation_type"], columns=data["city"])

city,Amsterdam,Barcelona,Vienna
accommodation_type,,,
Apartment,31,300,457
Hotel,799,1264,869


In [15]:
pd.crosstab(index=data["date"], columns=data["city"])

city,Amsterdam,Barcelona,Vienna
date,,,
2017-DEC-holiday,290,420,338
2017-NOV-weekday,315,452,377
2017-NOV-weekend,125,393,256
2018-JUNE-weekend,100,299,355


In [16]:
data["lnprice"] = np.log(data["price"])

In [17]:
data = data.filter(
    [
        "hotel_id",
        "date",
        "city",
        "accommodation_type",
        "stars",
        "rating",
        "distance",
        "price",
        "lnprice",
    ]
)


In [19]:
data.to_csv(os.path.join("hotels_work.csv"), index=False)

In [20]:
data = data.loc[
    (data["stars"] >= 3)
    & (data["stars"] <= 4)
    & (data["accommodation_type"] == "Hotel")
    & (data["city"] == "Vienna")
]

In [21]:
data["date"].value_counts()

date
2017-NOV-weekday     207
2017-DEC-holiday     189
2018-JUNE-weekend    181
2017-NOV-weekend     125
Name: count, dtype: int64

In [22]:
data[["distance", "price", "lnprice"]].describe().round(2)

,distance,price,lnprice
count,702.00,702.00,702.00
mean,1.57,122.75,4.74
std,1.15,53.30,0.37
min,0.00,50.00,3.91
25%,0.80,86.00,4.45
50%,1.40,109.00,4.69
75%,1.90,144.00,4.97
max,6.60,491.00,6.20


In [23]:
data.groupby("date")["distance"].describe().round(3)

,count,mean,std,min,25%,50%,75%,max
date,,,,,,,,
2017-DEC-holiday,189.0,1.503,1.060,0.0,0.8,1.3,1.9,5.2
2017-NOV-weekday,207.0,1.530,1.162,0.0,0.8,1.3,1.9,6.6
2017-NOV-weekend,125.0,1.773,1.298,0.0,0.9,1.6,2.1,6.6
2018-JUNE-weekend,181.0,1.531,1.130,0.0,0.8,1.3,1.9,6.6


In [24]:
data.groupby("date")["price"].describe().round(3)

,count,mean,std,min,25%,50%,75%,max
date,,,,,,,,
2017-DEC-holiday,189.0,116.492,46.308,57.0,85.0,103.0,138.0,386.0
2017-NOV-weekday,207.0,109.976,42.221,50.0,82.0,100.0,129.5,383.0
2017-NOV-weekend,125.0,149.144,76.531,60.0,92.0,132.0,180.0,491.0
2018-JUNE-weekend,181.0,125.674,45.054,59.0,94.0,111.0,154.0,297.0


In [25]:
data.groupby("date")["lnprice"].describe().round(3)

,count,mean,std,min,25%,50%,75%,max
date,,,,,,,,
2017-DEC-holiday,189.0,4.697,0.335,4.043,4.443,4.635,4.927,5.956
2017-NOV-weekday,207.0,4.640,0.337,3.912,4.407,4.605,4.864,5.948
2017-NOV-weekend,125.0,4.902,0.438,4.094,4.522,4.883,5.193,6.196
2018-JUNE-weekend,181.0,4.776,0.334,4.078,4.543,4.710,5.037,5.694


In [37]:
# Add two columns for piecewise linear splines: <2 miles and 2–7 miles
data["lspline_1"] = data["distance"].apply(lambda x: min(x, 2))
data["lspline_2"] = data["distance"].apply(lambda x: max(0, min(x - 2, 5)))
data


,hotel_id,date,city,accommodation_type,stars,rating,distance,price,lnprice,lspline_<2,lspline_2-7,lspline_1,lspline_2
140450,21897,2017-NOV-weekday,Vienna,Hotel,4.0,3.9,1.7,81,4.394449,1.7,0.0,1.7,0.0
140454,21897,2017-DEC-holiday,Vienna,Hotel,4.0,3.9,1.7,97,4.574711,1.7,0.0,1.7,0.0
140457,21897,2018-JUNE-weekend,Vienna,Hotel,4.0,3.9,1.7,97,4.574711,1.7,0.0,1.7,0.0
140458,21897,2017-NOV-weekend,Vienna,Hotel,4.0,3.9,1.7,107,4.672829,1.7,0.0,1.7,0.0
140470,21901,2017-DEC-holiday,Vienna,Hotel,4.0,3.7,1.4,109,4.691348,1.4,0.0,1.4,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
144085,22407,2017-NOV-weekday,Vienna,Hotel,4.0,4.4,1.0,100,4.605170,1.0,0.0,1.0,0.0
144087,22408,2018-JUNE-weekend,Vienna,Hotel,3.0,3.2,1.4,87,4.465908,1.4,0.0,1.4,0.0
144089,22408,2017-NOV-weekend,Vienna,Hotel,3.0,3.2,1.4,87,4.465908,1.4,0.0,1.4,0.0
144090,22408,2017-DEC-holiday,Vienna,Hotel,3.0,3.2,1.4,68,4.219508,1.4,0.0,1.4,0.0


In [40]:
# Piecewise spline regression with a knot at distance = 2
model_spline = smf.ols('lnprice ~ lspline_1 + lspline_2', data=data).fit(cov_type="HC1")
print(model_spline.summary())

                            OLS Regression Results                            
Dep. Variable:                lnprice   R-squared:                       0.308
Model:                            OLS   Adj. R-squared:                  0.306
Method:                 Least Squares   F-statistic:                     142.6
Date:                Sun, 06 Jul 2025   Prob (F-statistic):           1.17e-52
Time:                        05:18:12   Log-Likelihood:                -162.19
No. Observations:                 702   AIC:                             330.4
Df Residuals:                     699   BIC:                             344.0
Df Model:                           2                                         
Covariance Type:                  HC1                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      5.1571      0.028    186.711      0.0

### Table 9.3 External validity – comparing dates

In [41]:
dates = [
    "2017-NOV-weekday",
    "2017-NOV-weekend",
    "2017-DEC-holiday",
    "2018-JUNE-weekend",
]

In [42]:
models = []
for date in dates:
    models.append(
        smf.ols(
            "llnprice ~ lspline_1 + lspline_2", data=data.loc[lambda x: x["date"] == date]
        ).fit(cov_type="HC1")
    )

PatsyError: Error evaluating factor: NameError: name 'llnprice' is not defined
    llnprice ~ lspline_1 + lspline_2
    ^^^^^^^^

In [27]:
models = []
for date in dates:
    models.append(
        smf.ols(
            "lnprice ~ lspline(distance,2)", data=data.loc[lambda x: x["date"] == date]
        ).fit(cov_type="HC1")
    )

PatsyError: Error evaluating factor: NameError: name 'lspline' is not defined
    lnprice ~ lspline(distance,2)
              ^^^^^^^^^^^^^^^^^^^

In [27]:
stargazer = Stargazer(models)
stargazer.covariate_order(
    ["lspline(distance, 2)[0]", "lspline(distance, 2)[1]", "Intercept"]
)
stargazer.rename_covariates(
    {
        "Intercept": "Constant",
        "lspline(distance, 2)[0]": "Distance spline <2",
        "lspline(distance, 2)[1]": "Distance spline 2–7",
    }
)
stargazer.custom_columns(dates, [1, 1, 1, 1])
stargazer.show_model_numbers(False)
HTML(stargazer.render_html())


### Table 9.4 External validity – comparing dates 2

In [28]:
data["hotelcount"] = data.groupby("hotel_id")["city"].transform("count")


In [29]:
data["hotelcount"].value_counts().sort_index()


1      9
2     64
3    237
4    392
Name: hotelcount, dtype: int64

In [30]:
models = []
for date in dates:
    models.append(
        smf.ols(
            "lnprice ~ lspline(distance,2)",
            data=data.loc[lambda x: (x["date"] == date) & (x["hotelcount"] == 4)],
        ).fit(cov_type="HC1")
    )


In [31]:
stargazer = Stargazer(models)
stargazer.covariate_order(
    ["lspline(distance, 2)[0]", "lspline(distance, 2)[1]", "Intercept"]
)
stargazer.rename_covariates(
    {
        "Intercept": "Constant",
        "lspline(distance, 2)[0]": "Distance spline <2",
        "lspline(distance, 2)[1]": "Distance spline 2–7",
    }
)
stargazer.custom_columns(dates, [1, 1, 1, 1])
stargazer.show_model_numbers(False)
HTML(stargazer.render_html())


### Table 9.5 External validity – comparing cities

In [32]:
data = pd.read_csv(os.path.join(data_out, "hotels_work.csv"), index_col=0)


In [33]:
data = data.loc[
    lambda x: (x["stars"] >= 3)
    & (x["stars"] <= 4)
    & (x["date"] == "2017-NOV-weekday")
    & (x["accommodation_type"] == "Hotel")
]


In [34]:
pd.crosstab(index=data["city"], columns=data["stars"])


stars,3.0,3.5,4.0
city,,,
Amsterdam,103,17,75
Barcelona,101,4,144
Vienna,82,14,111


In [35]:
data.groupby("stars")["distance"].describe()


,count,mean,std,min,25%,50%,75%,max
stars,,,,,,,,
3.0,286.0,1.360140,1.127060,0.1,0.6,1.0,1.80,6.6
3.5,35.0,1.654286,1.152578,0.1,0.9,1.6,2.05,5.3
4.0,330.0,1.381818,1.058173,0.0,0.6,1.1,1.80,6.0


In [36]:
data.groupby("price")["distance"].describe()


,count,mean,std,min,25%,50%,75%,max
price,,,,,,,,
50,1.0,3.90,NaN,3.9,3.900,3.90,3.900,3.9
51,1.0,0.50,NaN,0.5,0.500,0.50,0.500,0.5
52,2.0,2.25,1.202082,1.4,1.825,2.25,2.675,3.1
54,2.0,1.55,0.636396,1.1,1.325,1.55,1.775,2.0
56,1.0,3.70,NaN,3.7,3.700,3.70,3.700,3.7
...,...,...,...,...,...,...,...,...
312,1.0,0.50,NaN,0.5,0.500,0.50,0.500,0.5
324,1.0,0.30,NaN,0.3,0.300,0.30,0.300,0.3
342,1.0,0.30,NaN,0.3,0.300,0.30,0.300,0.3


In [37]:
data.groupby("price")["distance"].describe()


,count,mean,std,min,25%,50%,75%,max
price,,,,,,,,
50,1.0,3.90,NaN,3.9,3.900,3.90,3.900,3.9
51,1.0,0.50,NaN,0.5,0.500,0.50,0.500,0.5
52,2.0,2.25,1.202082,1.4,1.825,2.25,2.675,3.1
54,2.0,1.55,0.636396,1.1,1.325,1.55,1.775,2.0
56,1.0,3.70,NaN,3.7,3.700,3.70,3.700,3.7
...,...,...,...,...,...,...,...,...
312,1.0,0.50,NaN,0.5,0.500,0.50,0.500,0.5
324,1.0,0.30,NaN,0.3,0.300,0.30,0.300,0.3
342,1.0,0.30,NaN,0.3,0.300,0.30,0.300,0.3


In [38]:
cities = ["Vienna", "Amsterdam", "Barcelona"]


In [39]:
models = []
for city in cities:
    models.append(
        smf.ols(
            "lnprice ~ lspline(distance,2)", data=data.loc[lambda x: x["city"] == city]
        ).fit(cov_type="HC1")
    )


In [40]:
stargazer = Stargazer(models)
stargazer.covariate_order(
    ["lspline(distance, 2)[0]", "lspline(distance, 2)[1]", "Intercept"]
)
stargazer.rename_covariates(
    {
        "Intercept": "Constant",
        "lspline(distance, 2)[0]": "Distance spline <2",
        "lspline(distance, 2)[1]": "Distance spline 2–7",
    }
)
stargazer.custom_columns(cities, [1, 1, 1])
stargazer.show_model_numbers(False)
HTML(stargazer.render_html())


### Table 9.6 External validity – accommodation types

In [41]:
data = pd.read_csv(os.path.join(data_out, "hotels_work.csv"), index_col=0)


In [42]:
data = data.loc[
    lambda x: (x["stars"] >= 3)
    & (x["stars"] <= 4)
    & (x["date"] == "2017-NOV-weekday")
    & (x["city"] == "Vienna")
]


In [43]:
pd.crosstab(index=data["accommodation_type"], columns=data["stars"])


stars,3.0,3.5,4.0
accommodation_type,,,
Apartment,34,41,17
Hotel,82,14,111


In [44]:
data.groupby("stars")["distance"].describe()


,count,mean,std,min,25%,50%,75%,max
stars,,,,,,,,
3.0,116.0,1.850862,1.300468,0.1,0.90,1.55,2.30,6.9
3.5,55.0,1.372727,1.122242,0.1,0.35,1.40,1.75,5.1
4.0,128.0,1.303125,1.033908,0.0,0.50,1.00,1.70,4.8


In [45]:
data.groupby("price")["distance"].describe()


,count,mean,std,min,25%,50%,75%,max
price,,,,,,,,
50,1.0,3.90,NaN,3.9,3.900,3.90,3.900,3.9
52,1.0,3.10,NaN,3.1,3.100,3.10,3.100,3.1
54,2.0,1.55,0.636396,1.1,1.325,1.55,1.775,2.0
56,2.0,3.10,0.848528,2.5,2.800,3.10,3.400,3.7
58,2.0,1.40,0.000000,1.4,1.400,1.40,1.400,1.4
...,...,...,...,...,...,...,...,...
355,1.0,1.60,NaN,1.6,1.600,1.60,1.600,1.6
363,1.0,1.50,NaN,1.5,1.500,1.50,1.500,1.5
364,2.0,0.40,0.282843,0.2,0.300,0.40,0.500,0.6


In [46]:
data.groupby("lnprice")["distance"].describe()


,count,mean,std,min,25%,50%,75%,max
lnprice,,,,,,,,
3.912023,1.0,3.90,NaN,3.9,3.900,3.90,3.900,3.9
3.951244,1.0,3.10,NaN,3.1,3.100,3.10,3.100,3.1
3.988984,2.0,1.55,0.636396,1.1,1.325,1.55,1.775,2.0
4.025352,2.0,3.10,0.848528,2.5,2.800,3.10,3.400,3.7
4.060443,2.0,1.40,0.000000,1.4,1.400,1.40,1.400,1.4
...,...,...,...,...,...,...,...,...
5.872118,1.0,1.60,NaN,1.6,1.600,1.60,1.600,1.6
5.894403,1.0,1.50,NaN,1.5,1.500,1.50,1.500,1.5
5.897154,2.0,0.40,0.282843,0.2,0.300,0.40,0.500,0.6


In [47]:
accommodation_types = ["Hotel", "Apartment"]


In [48]:
models = []
for atype in accommodation_types:
    models.append(
        smf.ols(
            "lnprice ~ lspline(distance,2)",
            data=data.loc[lambda x: x["accommodation_type"] == atype],
        ).fit(cov_type="HC1")
    )


In [49]:
stargazer = Stargazer(models)
stargazer.covariate_order(
    ["lspline(distance, 2)[0]", "lspline(distance, 2)[1]", "Intercept"]
)
stargazer.rename_covariates(
    {
        "Intercept": "Constant",
        "lspline(distance, 2)[0]": "Distance spline <2",
        "lspline(distance, 2)[1]": "Distance spline 2–7",
    }
)
stargazer.custom_columns(accommodation_types, [1, 1])
stargazer.show_model_numbers(False)
HTML(stargazer.render_html())
